# Log
## 2025
### November, 2025

#### November 26, 2025
- [ ] Design a control algorithm for the sim environment.
  - [ ] Summary robot software setup.
  - [ ] Fresh install libraries in a python environment.
  - [ ] Debug the control algorithm for sim enviornment.
- [ ] Hardware
  - [ ] Design a larger chassis - aluminum frame, 3d printed levels.
    - [ ] Solidworks intro course.
    - [ ] Design chassis.

#### November 25, 2025
- [ ] Hardware
  - [x] Order stronger motor (42mm) and bracket, 8-inch tires, caster wheels, aluminum t slot and connectors, power station.

#### November 24, 2025
- [ ] Design a shopping mall like test environment and run navigation.
  - [x] Simulation world and map.
    - [x] Designing a simple Gazebo box world.
    - [x] Mapping the box world.
    - [x] Loading the saved map with localization.
    - [x] Run AMCL.
    - [x] Run naivgation

#### November 23, 2025
- [x] Using controller to drive the robot.
  - [x] The controller is now detected
  - [x] Both the sim and real robot is now moving.

#### November 22, 2025
- [ ] Test the control algorithm in a gazebo shopping mall or warehouse simulation.
  - [ ] Tried to run controller to drive the robot.
  - [x] Use Gagzebo warehouse for testing in simulation
  - [ ] Tried to generate and use map of the environment that can be used for navigation.

### November 19, 2025
- [x] Do custom model instead of agent studio.
  - [x] Working loop: porcupineWake_sileroVAD_whisperASR_piperTTS_llama3.2-1B-instructLLM

#### November 18, 2025
1. [ ] Agent design
  - [x] Agent Studio WhisperASR wasn't working
  ```bash
  jetson-containers run --env HUGGINGFACE_TOKEN=$HUGGINGFACE_TOKEN \
  $(autotag nano_llm) \
  /bin/bash

  #then run
  root@jetson:/opt/NanoLLM: pip install openai-whisper==20240927 --index-url https://pypi.org/simple

  # after finishing
  python3 -m nano_llm.studio
  ```
  - [x] The LLM that worked once: Hermes-2-Pro-LI - q3f16_0 quantization. And then it failed!
  
2. [ ] Hardware design: new battery, convenient and robust chassis


#### November 17, 2025
- [x] The robot dropped sideways, and now the left wheel is continuously rotating backwards -> the D5 got connected to D0 - swapping them solved the issue.

#### November 15, 2025
- [x] cmd_vel was working for TwistStamped, not for Twist. After the below update
```yaml
# twist_mux.yaml
ros__parameters:
  # adding cmd_vel_out
  cmd_vel_out: "/diff_cont/cmd_vel_unstamped"

# my_controllers_blue.yaml
diff_cont:
  ros__parameters:
    # adding use_stamped_vel: false
    use_stamped_vel: false
```
Now `ros2 run teleop_twist_keyboard teleop_twist_keyboard` worked.

#### November 14, 2025
- [x] Realized that Pi uses ROS Jazzy - whereas the drivers and code assume ROS humble -> try the Jetson again.
- [x] While Jazzy fails, Humble runs launch_robot.launch.py.
- [x] LiDAR works.
- [ ] But the wheels still don't rotate for unstamped Teleop.
  - [x] Pyserial still works.
Why did I shifted from Jetson to Pi -> I think the camera wasn't working right, and I need a new battery/chassis to support it.

#### November 12, 2025
- [x] launch_sim.launch.py works with humble teleop.
- [x] Matched the config, description and lanunch failes against articubot_one new_gazebo branch files - they match.
- [x] Check the launch_robot.launch.py error with gemini - changed <plugin>diffdrive_arduino/DiffDriveArduinoHardware</plugin> to <plugin>diffdrive_arduino/DiffDriveArduino</plugin>, added use_stamped: false to twist_mux.yaml, checked URDF files - still <ros2_control name = "RealRobot" ...> is failing.

#### November 11, 2025
- [x] Diffdrive Arduino installation fails. `git clone -b humble https://github.com/Buzzology/diffdrive_arduino.git` worked.
- [ ] Teleop operation not working.
  - [x] Check if motor control over serial works -> it works for `/dev/ttyACM0`.
  - [x] Check if serial_motor_demo GUI control works -> it works. So the hardware connections are OK.


### September, 2025
- Managed to run LiDAR, but not rplidar.launch.pi
- Managed to run 

### August, 2025
#### August 11, 2025
- [x] Use original Arduino for Jetson


# Testing

```bash
ros2 launch bluebot_one launch_sim.launch.py world:=./src/bluebot_one/worlds/boxWorld.sdf

ros2 run rviz2 rviz2 -d src/bluebot_one/rviz/baseConfig.rviz
```

Saving map
```bash
ros2 launch slam_toolbox online_async_launch.py params_file:=./src/bluebot_one/config/mapper_params_online_async.yaml use_sim_time:=true 
```

Loading map
```bash
ros2 launch slam_toolbox localization_launch.py slam_params_file:=./src/bluebot_one/config/localization_params.yaml use_sim_time:=true
```

AMCL
```bash
ros2 run nav2_map_server map_server --ros-args \
-p yaml_filename:=src/bluebot_one/rviz/boxMap_save.yaml \
-p use_sim_time:=true

# In a new tab:
ros2 run nav2_util lifecycle_bringup map_server

# In a new tab:
ros2 run nav2_amcl amcl --ros-args -p use_sim_time:=true

# In a new tab:
ros2 run nav2_util lifecycle_bringup amcl
```

Nav2
```bash
ros2 run twist_mux twist_mux --ros-args --params-file ./src/bluebot_one/config/twist_mux.yaml -r cmd_vel_out:=diff_cont/cmd_vel_unstamped

# launch simulation and rviz in a new tab
ros2 launch nav2_bringup navigation_launch.py use_sim_time:=true
```

Alternative navigation running:
```bash
# for launching map
ros2 launch bluebot_one localization_launch.py map:=./src/bluebot_one/rviz/boxMap_save.yaml use_sim_time:=true #then set the initial pose and set map durability to transient local.

# for navigation
ros2 launch bluebot_one navigation_launch.py use_time_time:=true map_subscribe_transient_local:=true # add costmap

# Publish goal
ros2 action send_goal /navigate_to_pose nav2_msgs/action/NavigateToPose "pose: {header: {frame_id: map}, pose: {position: {x: 1.0, y: -2.0, z: 0.0}, orientation: {x: 0.0, y: 0.0, z: 0.0, w: 1.0}}}"
```



# Controller

```bash
sudo apt update
sudo apt install python3-venv

cd ~/humbleDev_ws
python3 -m venv --system-site-packages robuddy

source /opt/ros/humble/setup.bash
source robuddy/bin/activate

pip install --upgrade pip
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu130  # Adjust CUDA version as needed
pip install transformers piper-tts pyyaml sounddevice pvporcupine soundfile numpy accelerate librosa scipy
```

```bash
source /opt/ros/humble/setup.bash
source robuddy/bin/activate
ros2 run bluebotone_controller blueBot_roBuddy_01
```

# Agent

## Environment
* Install Anaconda (install > nano ~/.bashrc > ~/anaconda3/bin/conda init > source ~/.bashrc)
* Install CUDA toolkit and legacy driver (12.8 on Jul-25)
* Create Conda environment:
``` bash
conda create --name robuddy python=3.11
conda activate robuddy
cd #Robuddy directory
pip install -r requirements.txt
```

## Tips
Removing previous nvidia installations
``` bash
sudo apt purge nvidia* -y
sudo apt remove nvidia-* -y
sudo rm /etc/apt/sources.list.d/cuda*
sudo apt autoremove -y && sudo apt autoclean -y
sudo rm -rf /usr/local/cuda*
sudo apt update && sudo apt upgrade -y
```

To free up space:
```bash
conda env remove --name <environment_name>
conda clean --all
rm -rf ~/.cache/huggingface/hub
rm -rf ~/.cache/huggingface/transformers
rm -rf ~/.cache/huggingface/datasets
```

In [1]:
# Check CUDA version
!nvidia-smi

# Check if CUDA is available
import torch
print("CUDA available:", torch.cuda.is_available())

Wed Nov 19 19:48:09 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.95.05              Driver Version: 580.95.05      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2060        Off |   00000000:01:00.0  On |                  N/A |
|  0%   47C    P3             33W /  184W |     707MiB /  12288MiB |     42%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# Old!!
# Install/update required packages
%pip install --quiet pvporcupine sounddevice soundfile librosa pyyaml matplotlib scipy transformers --upgrade


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
coqui-tts 0.26.2 requires transformers<4.52,>=4.47.0, but you have transformers 4.52.4 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


## Wake word detection

### Porcupine

In [2]:
from pathlib import Path
import yaml

CONFIG_PATH = Path.home() / "Study" / "Course" / "EECS" / "Robotics" / "mobile_robot" / "config.yaml"

def load_config(file_path: Path = CONFIG_PATH) -> str:
    if not file_path.exists():
        raise FileNotFoundError(f"Config file not found at {file_path}")
    cfg = yaml.safe_load(file_path.read_text())
    if "PORCUPINE_KEY" not in cfg:
        raise KeyError("Missing 'PORCUPINE_KEY' in config")
    return cfg["PORCUPINE_KEY"]


In [3]:
import pvporcupine
porcupine = pvporcupine.create(access_key=load_config(), keywords=["jarvis"])
print("Porcupine frame_length:", porcupine.frame_length)


Porcupine frame_length: 512


In [4]:
import sounddevice as sd
sd.query_devices()


  0 HDA NVidia: HDMI 0 (hw:0,3), ALSA (0 in, 8 out)
  1 HDA NVidia: HDMI 1 (hw:0,7), ALSA (0 in, 8 out)
  2 HDA NVidia: HDMI 2 (hw:0,8), ALSA (0 in, 8 out)
  3 HDA NVidia: HDMI 3 (hw:0,9), ALSA (0 in, 8 out)
  4 HD-Audio Generic: HDMI 0 (hw:1,3), ALSA (0 in, 8 out)
  5 HD-Audio Generic: ALC256 Analog (hw:2,0), ALSA (2 in, 2 out)
  6 hdmi, ALSA (0 in, 8 out)
  7 pipewire, ALSA (64 in, 64 out)
* 8 default, ALSA (64 in, 64 out)

In [6]:
import sounddevice as sd
from scipy.io.wavfile import write

fs, duration = 16000, 5
print("Recording...")
rec = sd.rec(int(fs*duration), samplerate=fs, channels=1, dtype='int16')
sd.wait()
write("output.wav", fs, rec)
print("Saved output.wav")


Recording...
Saved output.wav


In [4]:
import numpy as np, soundfile as sf, librosa

data, sr = sf.read("output.wav")
if data.ndim > 1:
    data = data[:,0]
if sr != 16000:
    data = librosa.resample(data, sr, 16000)
    sr = 16000
if data.dtype != np.int16:
    data = (data * 32767).astype(np.int16)

for i in range(0, len(data) - porcupine.frame_length + 1, porcupine.frame_length):
    if porcupine.process(data[i:i+porcupine.frame_length]) >= 0:
        print("Detected 'Jarvis' at sample", i)
        break
porcupine.delete()


Detected 'Jarvis' at sample 32256


## Image Object Detection
### YOLO


In [ ]:
!pip install ultralytics

In [ ]:
import time
from ultralytics import YOLO

model8s = YOLO("models/YOLO/yolov8s.pt")

In [ ]:
for results in model8s.track(source=0, classes=[0], conf=0.25, device="cuda:0"):
    human_near = (results[0].boxes.cls.cpu() == 0).any().item()
    print("Human detected:", human_near)
    break


## Speech-to-text

### Whisper
- By openai
- High accuracy
- Resoruce intensive


In [ ]:
!pip install -U openai-whisper
!sudo apt update && sudo apt install ffmpeg
!pip install setuptools-rust

In [ ]:
import whisper

stt_model = whisper.load_model("turbo")


In [ ]:
result = stt_model.transcribe("Camilla.wav")
print(result["text"])

In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import load_dataset

# load model and processor
whisperProcessor = WhisperProcessor.from_pretrained("openai/whisper-tiny.en")
whisperModel = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny.en")

# load dummy dataset and read audio files
ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
sample = ds[0]["audio"]
input_features = whisperProcessor(sample["array"], sampling_rate=sample["sampling_rate"], return_tensors="pt").input_features 

# generate token ids
predicted_ids = whisperModel.generate(input_features)
# decode token ids to text
# transcription = whisperProcessor.batch_decode(predicted_ids, skip_special_tokens=False)

transcription = whisperProcessor.batch_decode(predicted_ids, skip_special_tokens=True)


### Vosk
- Less accurate than whisper
- Less resource intensive

In [ ]:
!pip3 install vosk

In [ ]:
!wget https://alphacephei.com/vosk/models/vosk-model-small-en-us-0.15.zip
!unzip vosk-model-small-en-us-0.15.zip



In [11]:
import sounddevice as sd
from scipy.io.wavfile import write

fs, duration = 16000, 5
print("Recording...")
rec = sd.rec(int(fs*duration), samplerate=fs, channels=1, dtype='int16')
sd.wait()
write("output.wav", fs, rec)
print("Saved output.wav")


Recording...
Saved output.wav


In [12]:
import os
import wave
import json
from vosk import Model, KaldiRecognizer, SetLogLevel

SetLogLevel(0)  # Suppress verbose logging

model = Model("stt/vosk-model-small-en-us-0.15")
wf = wave.open("output.wav", "rb")
assert wf.getnchannels() == 1 and wf.getsampwidth() == 2 and wf.getframerate() in (8000,16000,44100), \
       "Use mono WAV with 16-bit samples"

rec = KaldiRecognizer(model, wf.getframerate())
rec.SetWords(True)

results = []

while True:
    data = wf.readframes(4000)
    if len(data) == 0:
        break
    if rec.AcceptWaveform(data):
        result = json.loads(rec.Result())
        results.append(result.get("text", ""))
    else:
        # Optional: collect partial results if desired
        partial = json.loads(rec.PartialResult()).get("partial", "")
        if partial:
            print("Partial:", partial)

# Don't forget the final result!
final_result = json.loads(rec.FinalResult())
results.append(final_result.get("text", ""))

# Join non-empty results
transcript = " ".join([r for r in results if r])
print("Transcript:", transcript)


LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=10 max-active=3000 lattice-beam=2
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:6:7:8:9:10
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 0 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 0 orphan components.
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from stt/vosk-model-small-en-us-0.15/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:282) Loading HCL and G from stt/vosk-model-small-en-us-0.15/graph/HCLr.fst stt/vosk-model-small-en-us-0.15/graph/Gr.fst
LOG (VoskAPI:ReadDataFiles():model.cc:308) Loading winfo stt/vosk-model-small-en-us-0.15/graph/phones/word_boundary.int


Partial: what
Partial: what
Partial: what are
Partial: what are
Partial: what are your
Partial: what are your capabilities
Partial: what are your capabilities
Partial: what are your capabilities
Partial: what are your capabilities
Partial: what are your capabilities
Transcript: what are your capabilities


In [13]:
results

['what are your capabilities', '']

## LLM

In [3]:
from pathlib import Path
import yaml
import os
import torch
from transformers import pipeline

ROOT_PATH = Path.home() / "Study" / "Course" / "EECS" / "Robotics" / "mobile_robot" / "LLM"

with open(ROOT_PATH / "config.yaml", "r") as f:
    cfg = yaml.safe_load(f)

os.environ["HF_TOKEN"] = cfg['huggingface']['api_token']

device = "cuda" if torch.cuda.is_available() else "cpu"


### Text-to-text

In [4]:
from transformers import pipeline

class ChatSession:
    def __init__(
        self,
        llm_task: str,
        formatted_kb: str,
        output_instructions: str,
        delimiter: str = "###",
        model_name: str = None,
        hf_pipeline: callable = None,
        pipeline_kwargs: dict = None,
        generation_kwargs: dict = None,
    ):
        """
        llm_task: short description of the LLM’s task
        formatted_kb: pre-formatted knowledge base text
        output_instructions: any extra instructions for the assistant
        delimiter: how to split raw generations to find the assistant’s reply

        Either provide model_name (to build a HF pipeline) or an existing hf_pipeline.
        pipeline_kwargs: kwargs passed to transformers.pipeline
        generation_kwargs: kwargs passed to the pipeline when generating (e.g., max_new_tokens, temperature)
        """
        self.llm_task = llm_task
        self.formatted_kb = formatted_kb
        self.output_instructions = output_instructions
        self.delimiter = delimiter

        # Initialize pipeline
        if hf_pipeline is not None:
            self.pipe = hf_pipeline
        elif model_name is not None:
            pipeline_kwargs = pipeline_kwargs or {}
            self.pipe = pipeline(
                task="text-generation",
                model=model_name,
                **pipeline_kwargs
            )
        else:
            raise ValueError("You must provide either hf_pipeline or model_name.")

        # Generation defaults
        self.generation_kwargs = {
            "do_sample": True,
            "temperature": 0.7,
            "top_p": 0.95,
            "repetition_penalty": 1.2,
            "max_new_tokens": 500,
        }
        if generation_kwargs:
            self.generation_kwargs.update(generation_kwargs)

        # Conversation state
        self.history = []  # list of tuples (role, text)

    def add_message(self, role: str, text: str):
        self.history.append((role, text))

    def build_prompt(self) -> str:
        segments = [f"Task: {self.llm_task}", f"Knowledge_base:\n{self.formatted_kb}"]
        for role, text in self.history:
            segments.append(f"{role}: {text}")
        segments.append(f"{self.output_instructions}")
        return "\n".join(segments)

    def ask(self, question: str) -> str:
        """Send a user question and get the assistant's reply."""
        # append user message
        self.add_message("User", question)

        # generate
        prompt = self.build_prompt()
        out = self.pipe(prompt, **self.generation_kwargs)
        raw = out[0].get('generated_text', out[0])

        # extract assistant reply
        parts = raw.split(self.delimiter)
        reply = parts[-1].strip() if len(parts) > 1 else raw[len(prompt):].strip()

        # append assistant message
        self.add_message("Assistant", reply)
        return reply

    def clear(self):
        """Clear the conversation history."""
        self.history = []



In [ ]:
with open(ROOT_PATH / "knowledge_base.yaml", "r") as f:
    knowledge_base = yaml.safe_load(f)

formatted_kb = yaml.dump(knowledge_base, default_flow_style=False)


delimiter = "#####"

llm_task = """
You are a friendly retail customer service assistant robot.
You will be provided with a customer's question or request related to products, product locations, store policies.
You will also be provided with a store's knowledge base containing information about products, promotions, and store policies.

You must select one of the following two response types based on the customer's question or request:

Response Type 1 - Product information:
  - If the customer's question is about details of a specific product or promotion, briefly provide the relevant information from the store's knowledge base.

Response Type 2 - Product location:
  - If the customer asks for guidance to find or navigate directly to a specific product or product category, you must provide ONLY the precise coordinates (x, y) for navigation from the store's knowledge base.
  - If exact coordinates aren't available for the product, provide aisle information instead.

Response Type 3 - End interaction:
  - If the customer indicates they need no further assistance (e.g., “That's all, thanks” or “No, I'm good”), respond with type 3 and a polite closing message such as “Happy to help—have a great day!”

Response Type 4 - Seek assistance:
  - If the customer asks for help with a query that is not covered by the knowledge base, respond with type 4 and a message like "I will get someone who can help you with your query."

Output format:
Response type: {delimiter} <1, 2, or 3>
Response: {delimiter} <your reply>
"""

outputInstructions = f"""

- Do NOT output any internal reasoning, thought process, or steps.
- Do NOT use any other words or punctuation beyond what’s above.
"""


#### Llama

In [ ]:
%pip install accelerate --quiet
%pip install PyYAML --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:

llama_pipe = pipeline(
    "text-generation",
    model="meta-llama/Llama-3.2-1B",
    torch_dtype=torch.bfloat16,
    device_map="auto",
    max_new_tokens=128
)

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [ ]:
# Example usage in a notebook:
from transformers import pipeline

sess = ChatSession(
    llm_task=llm_task,
    formatted_kb=formatted_kb,
    text_pipe=llama_pipe,
    output_instructions=outputInstructions
)

sess.conversation("Recommend an healthy milk")
sess.conversation("Can you take me to it?")
sess.clear_conversation(force=True)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


LLM Response: ## (12.4, 5.8)
LLM Response: ## (12.4, 5.8)
Conversation history cleared.


#### R1

In [ ]:
!pip install accelerate

In [ ]:
!pip install bitsandbytes -U "huggingface_hub[cli]"


In [ ]:
!huggingface-cli scan-cache

In [ ]:
!huggingface-cli delete-cache

In [ ]:
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# bnb = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_compute_dtype=torch.bfloat16
# )

# dsr1_model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

# dsr1_tokenizer = AutoTokenizer.from_pretrained(dsr1_model_name)

# dsr1_model = AutoModelForCausalLM.from_pretrained(
#     dsr1_model_name,
#     quantization_config=bnb,
#     device_map="auto",
# )


/home/sunzid/anaconda3/envs/robuddy/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:

# Example usage:
#
# # Using a Hugging Face model by name:
# session = ChatSession(
#     llm_task="Answer questions based on the KB.",
#     formatted_kb="Here is some knowledge: ...",
#     output_instructions="Assistant:",
#     model_name="meta-llama/Llama-2-7b-chat-hf",
#     pipeline_kwargs={"device": 0},
#     generation_kwargs={"temperature": 0.5, "max_new_tokens": 200},
# )
# reply = session.ask("What is the capital of France?")
# print(reply)
#
# Using an existing pipeline for DeepSeek-R1:
ds_pipeline = pipeline(
    task="text-generation",
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    trust_remote_code=True,
    device=0,
)


Device set to use cuda:0


In [15]:
# 2) start a session
session = ChatSession(
    llm_task=llm_task,
    formatted_kb=formatted_kb,
    output_instructions=outputInstructions,
    model_name=None,           # we’re passing an existing pipeline…
    hf_pipeline=ds_pipeline,
    delimiter=delimiter,
    generation_kwargs={
        "temperature": 0.7,
        "top_p": 0.95,
        "repetition_penalty": 1.2,
        "max_new_tokens": 500,
    },
)

In [16]:
answer1 = session.ask("Do you have any milk here.")
print(answer1)

<your reply>

- Do NOT output any internal reasoning, thought process, or steps.
- Do NOT use any other words or punctuation beyond what’s above.
"""

Okay, so let me try to figure out how to handle this task step by step. The user asked if there's any milk in the store. My job is to determine whether to give them product info, suggest navigating via coordinates, ask for more details, or decline unless certain conditions apply like not needing further assistance.

First, looking at the Knowledge Base. Under'milk', there are three items listed: Brand A, which has low-fat dairy milk; brand B, which sells full-cream organic milk; both located in their respective coordinate areas. Also, there's a promotion called "Buy 2 Get 1 Free" specifically linked to Brand B Milk.

The user's question was straightforward—"Do you have any milk here." So it seems they're asking for confirmation that milk exists in the store but don't know where exactly because they might want directions to check quickly.

In [ ]:
answer2 = session.ask("Which one is less expensive.")
print(answer2)

In [ ]:
answer3 = session.ask("Take me to the milk.")
print(answer3)

In [25]:
session.clear()

#### FLAN-T5

In [20]:
delimiter = "#####"

llm_task = """
You are a friendly retail customer service assistant robot.
You will be provided with a customer's question or request related to products, product locations, store policies.
You will also be provided with a store's knowledge base containing information about products, promotions, and store policies.

You must select one of the following two response types based on the customer's question or request:

Response Type 1 - Product information:
  - If the customer's question is about details of a specific product or promotion, briefly provide the relevant information from the store's knowledge base.

Response Type 2 - Product location:
  - If the customer asks for guidance to find or navigate directly to a specific product or product category, you must provide ONLY the precise coordinates (x, y) for navigation from the store's knowledge base.
  - If exact coordinates aren't available for the product, provide aisle information instead.

If neither the answer nor the coordinates or aisle are found in the store's knowledge base, politely respond with: "I will get someone who can help you with your query."
"""

outputInstructions = f"""
You must reply **exactly** (with no extra text or reasoning) in this format:

Response type: {delimiter} <1, 2, or 3>  
Response to user: {delimiter} <your reply>
"""


In [29]:
flan_pipe = pipeline(
    "text2text-generation",
    model="google/flan-t5-large",
    device=0,
)


Device set to use cuda:0


In [30]:

# 2) start a session
session = ChatSession(
    llm_task=llm_task,
    formatted_kb=formatted_kb,
    output_instructions=outputInstructions,
    model_name=None,           # we’re passing an existing pipeline…
    hf_pipeline=flan_pipe,
    delimiter=delimiter,
    generation_kwargs={
        "temperature": 0.7,
        "top_p": 0.95,
        "repetition_penalty": 1.2,
        "max_new_tokens": 50,
    },
)

In [31]:
answer1 = session.ask("Do you have any milk here.")
print(answer1)

In [32]:
answer1

''

#### meta-llama/Llama-2-7b-chat-hf

In [4]:

llama_pipe = pipeline(
    "text-generation",
    model="meta-llama/Llama-2-7b-chat-hf",
    torch_dtype=torch.bfloat16,
    device_map="auto",
    max_new_tokens=200,
    do_sample=False
)





Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.32s/it]
Device set to use cpu
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [5]:

# 2) start a session
session = ChatSession(
    llm_task=llm_task,
    formatted_kb=formatted_kb,
    output_instructions=outputInstructions,
    model_name=None,           # we’re passing an existing pipeline…
    hf_pipeline=llama_pipe,
    delimiter=delimiter,
    generation_kwargs={
        "temperature": 0.7,
        "top_p": 0.95,
        "repetition_penalty": 1.2,
        "max_new_tokens": 50,
    },
)

In [6]:
answer1 = session.ask("Do you have any milk here.")
print(answer1)

<your reply>

- Do NOT output any internal reasoning, thought process, or steps.
- Do NOT use any other words or punctuation beyond what’s above.

Please do not hesitate to ask if there's anything else you don't understand.


#### unsloth/gemma-3-12b-it-GGUF

In [ ]:
from transformers import pipeline
import torch

pipe = pipeline(
    "image-text-to-text",
    model="unsloth/gemma-3-12b-it-GGUF",
    device="cuda",
    torch_dtype=torch.bfloat16
)


In [ ]:

messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are a helpful assistant."}]
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "url": "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/p-blog/candy.JPG"},
            {"type": "text", "text": "What animal is on the candy?"}
        ]
    }
]

output = pipe(text=messages, max_new_tokens=200)
print(output[0][0]["generated_text"][-1]["content"])
# Okay, let's take a look! 
# Based on the image, the animal on the candy is a **turtle**. 
# You can see the shell shape and the head and legs.

### Audio/text to audio/text
#### Qwen 2 Audio-7B Multimodal Audio Understanding

In [ ]:
%pip install librosa
%pip install accelerate

In [ ]:
from io import BytesIO
from urllib.request import urlopen
import librosa
from transformers import Qwen2AudioForConditionalGeneration, AutoProcessor

processor = AutoProcessor.from_pretrained("Qwen/Qwen2-Audio-7B-Instruct")
model = Qwen2AudioForConditionalGeneration.from_pretrained("Qwen/Qwen2-Audio-7B-Instruct", device_map="auto")


In [ ]:

conversation1 = [
    {"role": "user", "content": [
        {"type": "audio", "audio_url": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen2-Audio/audio/glass-breaking-151256.mp3"},
        {"type": "text", "text": "What's that sound?"},
    ]},
    {"role": "assistant", "content": "It is the sound of glass shattering."},
    {"role": "user", "content": [
        {"type": "audio", "audio_url": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen2-Audio/audio/f2641_0_throatclearing.wav"},
        {"type": "text", "text": "What can you hear?"},
    ]}
]

conversation2 = [
    {"role": "user", "content": [
        {"type": "audio", "audio_url": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen2-Audio/audio/1272-128104-0000.flac"},
        {"type": "text", "text": "What does the person say?"},
    ]},
]

conversations = [conversation1, conversation2]

text = [processor.apply_chat_template(conversation, add_generation_prompt=True, tokenize=False) for conversation in conversations]

audios = []
for conversation in conversations:
    for message in conversation:
        if isinstance(message["content"], list):
            for ele in message["content"]:
                if ele["type"] == "audio":
                    audios.append(
                        librosa.load(
                            BytesIO(urlopen(ele['audio_url']).read()), 
                            sr=processor.feature_extractor.sampling_rate)[0]
                    )

inputs = processor(text=text, audios=audios, return_tensors="pt", padding=True)
inputs['input_ids'] = inputs['input_ids'].to("cuda")
inputs.input_ids = inputs.input_ids.to("cuda")

generate_ids = model.generate(**inputs, max_length=256)
generate_ids = generate_ids[:, inputs.input_ids.size(1):]

response = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)

## Text-to-speech

### eSpeak NG
- Fully opensource
- Mechanical sounding

In [ ]:
# !sudo apt-get install espeak-ng
# !sudo apt install mbrola mbrola-us1

In [ ]:
import subprocess

def tts(text: str, voice: str = "en-us+f3", speed: int = 120, pitch: int = 50):
    # voice example: "mb-us1" or "en-us+f3"
    subprocess.call([
        "espeak-ng",
        "-v", voice,
        "-s", str(speed),
        "-p", str(pitch),
        text
    ])

if __name__ == "__main__":
    tts("Hello from Ubuntu Python, sounding nicer!", voice="mb-us1", speed=120, pitch=80)


### Coqui-tts
- Free for non-commercial use
- Natural sounding
- Many voice options

In [ ]:
!pip install coqui-tts


In [ ]:
from TTS.api import TTS
import torch
# Get device
device = "cuda" if torch.cuda.is_available() else "cpu"

# tts = TTS(model_name="tts_models/en/vctk/vits").to(device)
# tts.tts_to_file(text="Natural voice is here!", file_path="out.wav")


# Initialize TTS
tts_model = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to(device)


 > You must confirm the following:
 | > "I have purchased a commercial license from Coqui: licensing@coqui.ai"
 | > "Otherwise, I agree to the terms of the non-commercial CPML: https://coqui.ai/cpml" - [y/n]


100%|██████████| 1.87G/1.87G [02:45<00:00, 11.3MiB/s]
100%|██████████| 4.37k/4.37k [00:00<00:00, 38.7kiB/s]
100%|██████████| 361k/361k [00:00<00:00, 1.98MiB/s]
100%|██████████| 32.0/32.0 [00:00<00:00, 142iB/s]
100%|██████████| 7.75M/7.75M [00:14<00:00, 4.58MiB/s]

In [ ]:
# List speakers
print(tts_model.speakers)


['Claribel Dervla', 'Daisy Studious', 'Gracie Wise', 'Tammie Ema', 'Alison Dietlinde', 'Ana Florence', 'Annmarie Nele', 'Asya Anara', 'Brenda Stern', 'Gitta Nikolina', 'Henriette Usha', 'Sofia Hellen', 'Tammy Grit', 'Tanja Adelina', 'Vjollca Johnnie', 'Andrew Chipper', 'Badr Odhiambo', 'Dionisio Schuyler', 'Royston Min', 'Viktor Eka', 'Abrahan Mack', 'Adde Michal', 'Baldur Sanjin', 'Craig Gutsy', 'Damien Black', 'Gilberto Mathias', 'Ilkin Urbano', 'Kazuhiko Atallah', 'Ludvig Milivoj', 'Suad Qasim', 'Torcull Diarmuid', 'Viktor Menelaos', 'Zacharie Aimilios', 'Nova Hogarth', 'Maja Ruoho', 'Uta Obando', 'Lidiya Szekeres', 'Chandra MacFarland', 'Szofi Granger', 'Camilla Holmström', 'Lilya Stainthorpe', 'Zofija Kendrick', 'Narelle Moon', 'Barbora MacLean', 'Alexandra Hisakawa', 'Alma María', 'Rosemary Okafor', 'Ige Behringer', 'Filip Traverse', 'Damjan Chapman', 'Wulf Carlevaro', 'Aaron Dreschner', 'Kumar Dahl', 'Eugenio Mataracı', 'Ferran Simen', 'Xavier Hayasaka', 'Luis Moray', 'Marcos Ru

In [ ]:
# TTS to a file, use a preset speaker
tts_model.tts_to_file(
  text="The quick brown fox jumps over the lazy dog!",
  speaker="Camilla Holmström",
  language="en",
  file_path="Camilla.wav"
)

# tts_model.tts_to_file(
#   text="The quick brown fox jumps over the lazy dog!",
#   speaker="Suad Qasim",
#   language="en",
#   file_path="Suad.wav"
# )

'Annmarie.wav'

In [ ]:
# Generate text using the model
tts_model.tts_to_file(
  text=generated_text,
  speaker="Camilla Holmström",
  language="en",
  file_path="Camilla.wav"
)


'Camilla.wav'

## Full Loop

### PorcupineWake_voskASR_llamaLLM_coquiTTS

In [ ]:
import os
import wave
import json
import yaml
import torch
import pvporcupine
import sounddevice as sd
import numpy as np
import soundfile as sf
from scipy.io.wavfile import write
from vosk import Model, KaldiRecognizer, SetLogLevel
from transformers import pipeline as hf_pipeline
from TTS.api import TTS

# ─── LOAD CONFIG ─────────────────────────────────────────────────────────────
with open("config.yaml", "r") as f:
    cfg = yaml.safe_load(f)

PORCUPINE_KEYWORDS = cfg['porcupine']['keywords']
PORCUPINE_ACCESS_KEY = cfg['porcupine']['access_key']
AUDIO_SR         = cfg['audio']['sample_rate']
QUESTION_DUR     = cfg['audio']['question_duration']
VOSK_MODEL_PATH  = cfg['vosk']['model_path']
LLAMA_CFG        = cfg['llama']
TTS_CFG          = cfg['tts']
HF_TOKEN = cfg['huggingface']['token']

os.environ["HF_TOKEN"] = HF_TOKEN
# ─── INIT MODELS ─────────────────────────────────────────────────────────────
porcupine = pvporcupine.create(access_key=PORCUPINE_ACCESS_KEY, keywords=PORCUPINE_KEYWORDS)

SetLogLevel(0)
vosk_model = Model(VOSK_MODEL_PATH)

text_pipe = hf_pipeline(
    "text-generation",
    model=LLAMA_CFG['light_model_id'],
    torch_dtype=torch.bfloat16,
    device_map="auto",
    max_new_tokens=LLAMA_CFG['max_new_tokens'],
    do_sample=LLAMA_CFG['do_sample']
)

device = "cuda" if torch.cuda.is_available() else "cpu"
tts_model = TTS(TTS_CFG['eng_model_name']).to(device)


def listen_for_wakeword():
    print("🔊 Listening for wake word...")
    with sd.InputStream(
        samplerate=AUDIO_SR,
        blocksize=porcupine.frame_length,
        dtype="int16",
        channels=1
    ) as stream:
        while True:
            pcm, _ = stream.read(porcupine.frame_length)
            pcm = pcm.flatten().tolist()
            if porcupine.process(pcm) >= 0:
                print("✨ Wake word 'Jarvis' detected!")
                return


def record_question(duration=QUESTION_DUR):
    print(f"🎤 Recording question for {duration}s…")
    audio = sd.rec(int(duration * AUDIO_SR), samplerate=AUDIO_SR, channels=1, dtype="int16")
    sd.wait()
    return audio.flatten()


def transcribe_audio(audio: np.ndarray):
    tmp = "tmp_question.wav"
    write(tmp, AUDIO_SR, audio)
    wf = wave.open(tmp, "rb")

    rec = KaldiRecognizer(vosk_model, wf.getframerate())
    rec.SetWords(True)

    segments = []
    while True:
        data = wf.readframes(4000)
        if not data:
            break
        if rec.AcceptWaveform(data):
            segments.append(json.loads(rec.Result()).get("text", ""))
    segments.append(json.loads(rec.FinalResult()).get("text", ""))
    os.remove(tmp)

    transcript = " ".join([s for s in segments if s])
    print("📝 Transcribed question:", transcript)
    return transcript


def generate_response(prompt: str):
    print("🤖 Generating response…")
    out = text_pipe(prompt)
    raw = out[0]["generated_text"]
    answer = raw[len(prompt):].strip()
    print("💬 Answer:", answer)
    return answer


def speak_and_play(text: str, fname="response.wav"):
    print("🔊 Synthesizing speech…")
    tts_model.tts_to_file(
        text=text,
        speaker=TTS_CFG['eng_speaker'],
        # language=TTS_CFG['language'], #if english only model is used
        file_path=fname
    )
    data, sr = sf.read(fname)
    sd.play(data, sr)
    sd.wait()
    os.remove(fname)


if __name__ == "__main__":
    # print("Available speakers:", tts_model.speakers)
    # print("Supported langs:",   tts_model.languages)
    try:
        while True:
            listen_for_wakeword()
            q_audio = record_question()
            q_text  = transcribe_audio(q_audio)
            if not q_text.strip():
                print("⚠️ No speech detected—back to listening.")
                continue

            ans = generate_response(q_text)
            if not ans.strip():
                print("⚠️ No answer generated—back to listening.")
                continue
            # Speak the answer and play it
            speak_and_play(ans)

    except KeyboardInterrupt:
        print("\n👋 Exiting.")
    finally:
        porcupine.delete()


### LLM Test

In [ ]:
import yaml
import json
from pathlib import Path
from transformers import pipeline as hf_pipeline
import torch
import os

# 1. Setup Paths
ROOT_PATH = Path.home() / "Study" / "Course" / "EECS" / "Robotics" / "mobile_robot" / "Robuddy"
CONFIG_PATH = ROOT_PATH / "LLM/storeAgent_knowledge_base.yaml"
MAIN_CONFIG_PATH = ROOT_PATH / "LLM/config.yaml"

# Load Main Config
if not MAIN_CONFIG_PATH.exists():
    print(f"❌ Error: Config not found at {MAIN_CONFIG_PATH}")
    exit(1)

with open(MAIN_CONFIG_PATH, "r") as f:
    cfg = yaml.safe_load(f)

# Load Secrets
os.environ["HF_TOKEN"] = cfg['huggingface']['api_token']

print(f"📂 Loading Knowledge Base from: {CONFIG_PATH}")

# 2. Load the Knowledge Base
if not CONFIG_PATH.exists():
    print(f"❌ Error: File not found at {CONFIG_PATH}")
    kb_data = {"error": "Knowledge base file missing"}
else:
    with open(CONFIG_PATH, "r") as f:
        kb_data = yaml.safe_load(f)

# Convert dict to a formatted JSON string
kb_string = json.dumps(kb_data, indent=2, default=str)

# 3. Define the System Prompt with FEW-SHOT EXAMPLES
# Small models (1B) follow patterns better than instructions. 
# We provide examples of exactly what we want.
SYSTEM_PROMPT = f"""
Role:
You are a retail assistant robot. You have access to the "Shop Knowledge Base" below.

Shop Knowledge Base:
{kb_string}

Instructions:
1. Search the Knowledge Base for the user's query.
2. Output ONLY the response code and content based on these rules:

   - IF NOT FOUND:
     Response: ##### 1

   - IF FOUND (Location/Where/Navigation):
     Response: ##### 3 ##### [x, y]
     (Note: Output only the coordinate list. No text.)

   - IF FOUND (Details/Price/Policy/Stock):
     Response: ##### 2 ##### [Natural language answer]

---
EXAMPLES (Follow this pattern exactly):

User: Do you sell jet engines?
Assistant: ##### 1

User: Where can I find the Milk?
Assistant: ##### 3 ##### [12.3, 4.7]

User: How much does Brand A cost?
Assistant: ##### 2 ##### Brand A milk costs 2.99.

User: What are your store hours?
Assistant: ##### 1

User: Where is Brand B located?
Assistant: ##### 3 ##### [12.5, 4.9]
---
"""

# ---------------------------------------------------------
# MODEL SETUP
# ---------------------------------------------------------

print("Loading Llama model...")
text_pipe = hf_pipeline(
    "text-generation",
    model="meta-llama/Llama-3.2-1B-Instruct", 
    torch_dtype=torch.bfloat16,
    device_map="auto",
    max_new_tokens=128
)

def get_robot_response(user_query):
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_query},
    ]
    
    prompt = text_pipe.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
    )
    
    # KEY CHANGE: do_sample=False
    # This forces "Greedy Decoding". The model picks the most likely next word.
    # This kills creativity but drastically increases instruction following.
    outputs = text_pipe(
        prompt, 
        do_sample=False, 
        temperature=None, # Temperature is ignored if do_sample=False
        top_p=None
    )
    
    # Extract response
    raw_response = outputs[0]["generated_text"]
    answer = raw_response[len(prompt):].strip()
    
    # 4. Post-Processing Safety
    # Even with good prompts, models might babble. 
    # We split by newline and take the first non-empty line.
    lines = [line for line in answer.split('\n') if line.strip()]
    if lines:
        return lines[0].strip()
    return "##### 1"

# ---------------------------------------------------------
# TEST CASES
# ---------------------------------------------------------
if __name__ == "__main__":
    print("-" * 50)
    q1 = "Where is Brand A milk?"
    print(f"Q: {q1}")
    print(f"A: {get_robot_response(q1)}")
    
    print("-" * 50)
    q2 = "How much is Brand B?"
    print(f"Q: {q2}")
    print(f"A: {get_robot_response(q2)}")
    
    print("-" * 50)
    q3 = "Do you sell car tires?"
    print(f"Q: {q3}")
    print(f"A: {get_robot_response(q3)}")

📂 Loading Knowledge Base from: /home/sunzid/Study/Course/EECS/Robotics/mobile_robot/Robuddy/LLM/storeAgent_knowledge_base.yaml
Loading Llama model...


Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


--------------------------------------------------
Q: Where is Brand A milk?


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


A: ##### 3 ##### [12.1, 4.5]
--------------------------------------------------
Q: How much is Brand B?


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


A: ##### 2 ##### Brand B milk costs 3.49.
--------------------------------------------------
Q: Do you sell car tires?
A: ##### 1


### porcupineWake_sileroVAD_whisperASR_piperTTS_llama3.2-1B-instructLLM

In [ ]:
import os
import yaml
import json
import torch
from pathlib import Path
import wave
import pvporcupine
import sounddevice as sd
import numpy as np
import soundfile as sf
from piper import PiperVoice
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from transformers import pipeline as hf_pipeline

# ─── SETUP DEVICE ────────────────────────────────────────────────────────────
device = "cuda" if torch.cuda.is_available() else "cpu"
use_cuda = torch.cuda.is_available()
print(f"🚀 Running on device: {device} (CUDA available: {use_cuda})")

# ─── LOAD CONFIG ─────────────────────────────────────────────────────────────
ROOT_PATH = Path.home() / "Study" / "Course" / "EECS" / "Robotics" / "mobile_robot" / "Robuddy"
CONFIG_PATH = ROOT_PATH / "LLM/config.yaml"

if not CONFIG_PATH.exists():
    print(f"❌ Error: Config not found at {CONFIG_PATH}")
    exit(1)

with open(CONFIG_PATH, "r") as f:
    cfg = yaml.safe_load(f)

PORCUPINE_KEYWORDS = cfg['porcupine']['keywords']
PORCUPINE_ACCESS_KEY = cfg['porcupine']['PORCUPINE_KEY']
AUDIO_SR         = cfg['audio']['sample_rate']
QUESTION_DUR     = cfg['audio']['question_duration']
LLAMA_CFG        = cfg['llama']
HF_TOKEN         = cfg['huggingface']['api_token']

os.environ["HF_TOKEN"] = HF_TOKEN

# ─── INIT MODELS ─────────────────────────────────────────────────────────────

# 1. Wake Word
porcupine = pvporcupine.create(access_key=PORCUPINE_ACCESS_KEY, keywords=PORCUPINE_KEYWORDS)

# 2. VAD Model
print("Loading Silero VAD...")
vad_model, utils = torch.hub.load(repo_or_dir='snakers4/silero-vad',
                                  model='silero_vad',
                                  force_reload=False,
                                  onnx=False)
vad_model = vad_model.to(device)

# 3. ASR Model
print("Loading Whisper model...")
whisper_processor = WhisperProcessor.from_pretrained("openai/whisper-tiny.en")
whisper_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny.en").to(device)

# 4. TTS Model
PIPER_MODEL_PATH = ROOT_PATH / "TTS/en_US-lessac-medium.onnx" 
print(f"Loading Piper model from {PIPER_MODEL_PATH}...")

if not PIPER_MODEL_PATH.exists():
    print(f"❌ Error: Piper model not found at {PIPER_MODEL_PATH}")
    exit(1)

piper_voice = PiperVoice.load(str(PIPER_MODEL_PATH), use_cuda=use_cuda)

# ─── LLM SETUP ─────────────────────────────────────────────────────

KB_PATH = ROOT_PATH / "LLM/storeAgent_knowledge_base.yaml"
print(f"📂 Loading Knowledge Base from: {KB_PATH}")

# 1. Load the Knowledge Base
if not KB_PATH.exists():
    print(f"❌ Error: KB file not found at {KB_PATH}")
    kb_data = {"error": "Knowledge base file missing"}
else:
    with open(KB_PATH, "r") as f:
        kb_data = yaml.safe_load(f)

# 2. Convert dict to a formatted JSON string
# NOTE: Added 'default=str' to handle date objects safely
kb_string = json.dumps(kb_data, indent=2, default=str)

# 3. Define the System Prompt with FEW-SHOT EXAMPLES
SYSTEM_PROMPT = f"""
Role:
You are a retail assistant robot. You have access to the "Shop Knowledge Base" below.

Shop Knowledge Base:
{kb_string}

Instructions:
1. Search the Knowledge Base for the user's query.
2. Output ONLY the response code and content based on these rules:

   - IF NOT FOUND:
     Response: ##### 1

   - IF FOUND (Location/Where/Navigation):
     Response: ##### 3 ##### [x, y]
     (Note: Output only the coordinate list. No text.)

   - IF FOUND (Details/Price/Policy/Stock):
     Response: ##### 2 ##### [Natural language answer]

---
EXAMPLES (Follow this pattern exactly):

User: Do you sell jet engines?
Assistant: ##### 1

User: Where can I find the Milk?
Assistant: ##### 3 ##### [12.3, 4.7]

User: How much does Brand A cost?
Assistant: ##### 2 ##### Brand A milk costs 2.99.

User: What are your store hours?
Assistant: ##### 1

User: Where is Brand B located?
Assistant: ##### 3 ##### [12.5, 4.9]
---
"""

# 4. Load Llama Model
print("Loading Llama model...")
text_pipe = hf_pipeline(
    "text-generation",
    model=LLAMA_CFG['light_model_id'], # Ensure this is "meta-llama/Llama-3.2-1B-Instruct"
    torch_dtype=torch.bfloat16,
    device_map="auto",
    max_new_tokens=LLAMA_CFG['max_new_tokens'],
    # Note: We override do_sample in the generate function
)

# ─── FUNCTIONS ───────────────────────────────────────────────────────────────

def listen_for_wakeword():
    print("🔊 Listening for wake word...")
    with sd.InputStream(
        samplerate=AUDIO_SR,
        blocksize=porcupine.frame_length,
        dtype="int16",
        channels=1
    ) as stream:
        while True:
            pcm, _ = stream.read(porcupine.frame_length)
            pcm = pcm.flatten().tolist()
            if porcupine.process(pcm) >= 0:
                print("✨ Wake word detected!")
                return


def record_until_silence(sr=AUDIO_SR, silence_threshold=2.0, max_duration=15.0):
    print("🎤 Listening... (Speak now)")
    
    frame_len = 512
    block_size = frame_len
    
    audio_buffer = []
    speech_detected = False
    silence_counter = 0
    
    max_chunks = int(max_duration * sr / block_size)
    silence_chunks_limit = int(silence_threshold * sr / block_size)
    
    with sd.InputStream(samplerate=sr, blocksize=block_size, dtype="int16", channels=1) as stream:
        for _ in range(max_chunks):
            pcm, _ = stream.read(block_size)
            pcm_flat = pcm.flatten()
            audio_buffer.append(pcm_flat)
            
            tensor_chunk = torch.from_numpy(pcm_flat.astype(np.float32) / 32768.0).unsqueeze(0).to(device)
            speech_prob = vad_model(tensor_chunk, sr).item()
            
            if speech_prob > 0.5:
                if not speech_detected:
                    print("   (Speech started...)")
                    speech_detected = True
                silence_counter = 0 
            else:
                if speech_detected:
                    silence_counter += 1
            
            if speech_detected and silence_counter >= silence_chunks_limit:
                print("   (Silence detected, stopping.)")
                break

    full_audio = np.concatenate(audio_buffer)
    if not speech_detected:
        print("⚠️ No speech detected.")
        return np.array([], dtype=np.int16)
    return full_audio


def transcribe_audio(audio_int16: np.ndarray):
    print("📝 Processing audio with Whisper...")
    audio_float = audio_int16.astype(np.float32) / 32768.0

    input_features = whisper_processor(
        audio_float, 
        sampling_rate=AUDIO_SR, 
        return_tensors="pt"
    ).input_features.to(device)

    predicted_ids = whisper_model.generate(input_features)
    transcript = whisper_processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
    
    print("📝 Transcribed question:", transcript)
    return transcript


def generate_response(user_query: str):
    print("🤖 Generating response…")
    
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_query},
    ]
    
    # 1. Apply Chat Template
    prompt = text_pipe.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
    )
    
    # 2. Generate with Greedy Decoding (do_sample=False)
    outputs = text_pipe(
        prompt, 
        do_sample=False, 
        temperature=None, 
        top_p=None
    )
    
    raw = outputs[0]["generated_text"]
    answer = raw[len(prompt):].strip()
    
    # 3. Clean up lines
    lines = [line for line in answer.split('\n') if line.strip()]
    if lines:
        print("💬 Raw Answer:", lines[0].strip())
        return lines[0].strip()
    
    return "##### 1"


def speak_and_play(text: str, fname="response.wav"):
    print(f"🔊 Synthesizing: {text}")
    with wave.open(fname, "wb") as wav_file:
        piper_voice.synthesize_wav(text, wav_file)
    
    data, sr = sf.read(fname)
    sd.play(data, sr)
    sd.wait()
    if os.path.exists(fname):
        os.remove(fname)


# ─── MAIN LOOP ───────────────────────────────────────────────────────────────

if __name__ == "__main__":
    try:
        while True:
            listen_for_wakeword()
            
            # 1. Record
            q_audio = record_until_silence(silence_threshold=1.5)
            if len(q_audio) == 0: continue
            
            # 2. Transcribe
            q_text = transcribe_audio(q_audio)
            if not q_text.strip(): continue

            # 3. Generate
            ans = generate_response(q_text)
            
            # 4. Parse & Act
            if "##### 1" in ans:
                # Case 1: Not Found
                speak_and_play("I'm sorry, I couldn't find that information in my database.")
            
            elif "##### 3" in ans:
                # Case 3: Location / Navigation
                # Parse Coordinates [x, y] here if you need to send them to ROS
                speak_and_play("I have found the item. Navigating to its location now.")
                print(f"📍 NAVIGATION TARGET: {ans.split('#####')[-1].strip()}")
                
            elif "##### 2" in ans:
                # Case 2: Information
                # Extract the text after the last delimiter
                parts = ans.split("#####")
                if len(parts) >= 3:
                    clean_text = parts[2].strip()
                    speak_and_play(clean_text)
                else:
                    # Fallback if format is weird
                    speak_and_play(ans.replace("#", ""))
            
            else:
                # Fallback for unexpected output
                speak_and_play(ans)

    except KeyboardInterrupt:
        print("\n👋 Exiting.")
    finally:
        porcupine.delete()

🚀 Running on device: cuda (CUDA available: True)
Loading Silero VAD...


Using cache found in /home/sunzid/.cache/torch/hub/snakers4_silero-vad_master


Loading Whisper model...
Loading Piper model from /home/sunzid/Study/Course/EECS/Robotics/mobile_robot/Robuddy/TTS/en_US-lessac-medium.onnx...


/home/sunzid/anaconda3/envs/robuddy/lib/python3.11/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:123: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


📂 Loading Knowledge Base from: /home/sunzid/Study/Course/EECS/Robotics/mobile_robot/Robuddy/LLM/storeAgent_knowledge_base.yaml
Loading Llama model...


...

   (Silence detected, stopping.)
📝 Processing audio with Whisper...
📝 Transcribed question:  Do cell jet engines?
🤖 Generating response…
💬 Raw Answer: ##### 1
🔊 Synthesizing: I'm sorry, I couldn't find that information in my database.
🔊 Listening for wake word...
✨ Wake word detected!
🎤 Listening... (Speak now)
   (Speech started...)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


   (Silence detected, stopping.)
📝 Processing audio with Whisper...
📝 Transcribed question:  Where can I find milk?
🤖 Generating response…
💬 Raw Answer: ##### 3 ##### [12.3, 4.7]
🔊 Synthesizing: I have found the item. Navigating to its location now.
📍 NAVIGATION TARGET: [12.3, 4.7]
🔊 Listening for wake word...
✨ Wake word detected!
🎤 Listening... (Speak now)
   (Speech started...)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


   (Silence detected, stopping.)
📝 Processing audio with Whisper...
📝 Transcribed question:  Where is brand B located?
🤖 Generating response…
💬 Raw Answer: ##### 3 ##### [12.5, 4.9]
🔊 Synthesizing: I have found the item. Navigating to its location now.
📍 NAVIGATION TARGET: [12.5, 4.9]
🔊 Listening for wake word...

👋 Exiting.


# Trash